<center>
    <h1>Verbal Explanation of Spatial Temporal GNNs for Traffic Forecasting</h1>
    <h2>Monte Carlo Tree Search on the PeMS-Bay dataset</h2>
</center>

---

In this notebook the explanations of the predicted events of the *STGNN* on the *PeMS-Bay* dataset are computed by means of an *Explainer*. The explainer is an MCTS algorithm that performs a localized search on the input graph of the instance to be explained. It returns the important subgraph which serves as the explanation of the predicted selected event outputed by the STGNN. The search space is cut by a transparent *global heuristic* that exploits traffic domain-specific characteristics and limits the search on a subgraph of the input graph composed of the top scoring nodes according to the global heuristic. The explainer is inspired by the paper *Explaining temporal graph models through an explorer-navigator framework* <a name="cite_paper"></a>[<sup>[1]</sup>](#note_paper).

Firstly, grid search is applied to the training set in order to tune the hyperparameters of the explainer, afterwards, the important subgraphs of each dataset are extracted and the results are evaluated.

For more detailed informations about the used functions, look into the corresponding docstrings inside the python files, inside the `src` folder.

---
<small>

<a name="note_paper"></a>[1]
W. Xia et al. “Explaining temporal graph models through an explorer-navigator framework”. In: *The
Eleventh International Conference on Learning Representations*, 2023.
URL: https://openreview.net/forum?id=BR_ZhvcYbGJ.
</small>

In [4]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [5]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

In [7]:
import torch

# Set the device for training and querying the model.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The selected device is: "{DEVICE}"')

The selected device is: "cuda"


# 1 Loading the Data
In this section the data is loaded

In [8]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'pems-bay')

In [9]:
import pickle
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [10]:
from src.spatial_temporal_gnn.model import SpatialTemporalGNN
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_pems_bay.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

# Get the STGNN and load the checkpoints.
spatial_temporal_gnn = SpatialTemporalGNN(9, 1, 12, 12, adj_matrix, DEVICE, 64)

stgnn_checkpoints_path = os.path.join('..', 'models', 'checkpoints',
                                      'st_gnn_pems_bay.pth')

stgnn_checkpoints = torch.load(stgnn_checkpoints_path)
spatial_temporal_gnn.load_state_dict(stgnn_checkpoints['model_state_dict'])

# Set the STGNN in evaluation mode.
spatial_temporal_gnn.eval();

In [11]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_pems_bay.csv'),
    has_header=False)

In [12]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [13]:
import pickle

# Get the data scaler.
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [14]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train.npy'))
y_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train.npy'))
x_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train_time.npy'))
y_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train_time.npy'))

x_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val.npy'))
y_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val.npy'))
x_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val_time.npy'))
y_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val_time.npy'))

x_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test.npy'))
x_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test_time.npy'))
y_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test_time.npy'))

# 2 Explainer
In this section the explainer algorithm is applied through a grid search on the train dataset, then the results are evaluated.

The explainer firstly computes a global heuristic $c_{v_i, v_j}^t$ among all input nodes $v_i$ at timesteps $t$ and the nodes of the event to explain $v_j$ as:
$$ c_{v_i, v_j}^t = \frac{\Delta t}{\Delta d} + \frac{\exp (- (\Delta t + \Delta d))}{\Delta s} $$
Where $\Delta t$ is the temporal distance between $t$ and the last timestep of $v_j$, $\Delta d$ their spatial distance and $\Delta s$ the abdolute difference between the speed of $v_i$ at time $t$ and the average speed of $v_j$ in the event to explain.

Afterwards, it performs a MCTS using the input subgraph composed of top scoring nodes by the heuristic as the root of the tree. It performs a series of rollout that involve choosing a path from the root to a leaf node. At each branching, a node of the graph described in the tree node is removed and a child tree node is obtained. The leaf node respects a sparsity threshold, as its graph has a limited pre-defined number of nodes. In practice, given node of the tree $\mathcal{N}_i$:
    
1. A new tree node is expanded from $\mathcal{N}_i$ according to an action. The action consists in removing a specific traffic node in the graph described in $\mathcal{N}_i$.
2. A child node is selected from $\mathcal{N}_i$ among the expanded ones. In particular, it is selected the child node that maximizes exploration and exploitation.
3. If $\mathcal{N}_i$ is a leaf node, its reward is computed using the STGNN $\Phi$.
4. If $\mathcal{N}_i$ is a leaf node, its reward is backpropagated from it up to the path nodes until the root is reached to update information useful for exploration and exploitation in the tree search.

Ultimately, the final explanation outcome defined as the important subgraph is determined by the subgraph of the leaf node that attains the highest reward and which meets the specified sparsity threshold.

## 2.1 Spatial Distance Matrix
Firstly, a speed distance matrix is built in order to compute the shortest distance in kms for each pair of nodes. This matrix is built by firstly finding the shortest path between couple of nodes through the distance matrix $A$ and then computing the length of the path in kms.

Given a path $[n_1, ..., n_m]$, the distance among $n_1$ and $n_m$ is obtained as:
$$ \sum_{i}^{m -1} \text{geodesic}(n_i, n_{i+1})$$

In [12]:
from src.data.data_processing import get_distance_matrix

if not os.path.exists(
    os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy')):
    # Build the distance matrix between the nodes.
    distance_matrix = get_distance_matrix(
        adj_matrix,
        locations_df,
        node_pos_dict)

    # Save the distance matrix.
    np.save(os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'),
            distance_matrix)

else:
    # Load the distance matrix.
    distance_matrix = np.load(
        os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'))

## 2.2 Grid Search

Grid search is performed on a portion of the train dataset in order to find the best set of hyperparameters of the Explainer:
* `explanation size factor`: multiplied by the number of the nodes of the event to explain, defines the size of the important subgraph found by the explainer;
* `cut size factor`: multiplied by `explanation size factor`, defines the number of top scoring nodes of the input graph according to the global heuristic that have to be used to cut the search;
* `exploration weight`: parameter to weight the exploration in the MCTS;
* `n rollouts`: the number of simulations to perform in the MCTS.
* `remove value`: The value used to flag the missing data speed in the important subgraph. 

The results are evaluated in terms of *Fidelity*$^-$ and *average time* to output the subgraph.

* **Fidelity**$^\textbf{-}$
    $$
        \text{Fidelity}^- = \frac{1}{N}  \sum_{i=1}^N (\Phi(\mathcal{G}_i) - \Phi({\mathcal{G}}_i^m))
    $$
    with $N$ the number of instances, $\Phi$ the STGNN, $\mathcal{G}_i$ the $i^th$ input graph and $\mathcal{G}_i^m$ the important subgraph.
    The error $\Phi(\mathcal{G}_i) - \Phi({\mathcal{G}}_i^m)$ is computed using *MAE*, *RMSE* and *MAPE*.

* **Average time:**
    It measures the average time employed to output the important subgraph by for each instance.

In [ ]:
from src.explanation.monte_carlo.explanation import apply_grid_search

apply_grid_search(
    x_train[::10],
    y_train[::10],
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts_list=[30, 50],
    explanation_size_factor_list=[2, 3],
    cut_size_factor_list=[2],
    exploration_weight_list=[5, 10, 20],
    remove_value_list=['perturb', 0.])

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: perturb
[100/100] - 586s - MAE: { severe_congestion 3.98 -congestion 2.02 -free_flow 0.849 - total: 2.05 } - RMSE: { severe_congestion 4.45 -congestion 2.28 -free_flow 0.968 - total: 2.33 } - MAPE: { severe_congestion 16.9% -congestion 4.03% -free_flow 1.27% - total: 6.81% } - Average time: 5.86s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: 0.0
[100/100] - 582s - MAE: { severe_congestion 11.3 -congestion 2.7 -free_flow 1.69 - total: 5.24 } - RMSE: { severe_congestion 11.6 -congestion 3.17 -free_flow 1.91 - total: 5.57 } - MAPE: { severe_congestion 56.5% -congestion 5.53% -free_flow 2.51% - total: 21.7% } - Average time: 5.82s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 50 remove_value: perturb
[100/100] - 937s - MAE: { severe_congestion 3.31 -congestion 1.78 -free_flow 0.83

The best hyperparameters are expressed below.

In [15]:
CUT_SIZE_FACTOR = 2
EXPLANATION_SIZE_FACTOR = 2
EXPLORATION_WEIGHT = 20
N_ROLLOUTS = 30
REMOVE_VALUE = 'perturb'

## 2.3 Computing the explanations and Saving the Results
In this section the explanations of the predicted events of the train, validation and test datasets are computed using the selected hyperparameters.

The results are evaluated in terms of *Fidelity*$^-$ using *MAE*, *RMSE* and *MAPE*; *Average time*; *Fidelity*$^+$ using *MAE*, *RMSE* and *MAPE*; and *Sparsity*.

*Fidelity*$^-$ and *Average time* have already been introduced. For what concerns *Fidelity*$^+$ and *Sparsity*:

* **Fidelity**$^\textbf{+}$
    $$
        \text{Fidelity}^+ = \frac{1}{N}  \sum_{i=1}^N (\Phi(\mathcal{G}_i) - \Phi({\mathcal{G}}_i^{1 - m}))
    $$
    with $N$ the number of instances, $\Phi$ the STGNN, $\mathcal{G}_i$ the $i^th$ input graph and $\mathcal{G}_i^{1 - m}$ the complement of the important subgraph.
* **Sparsity:**
    $$\text{Sparsity} = \frac{1}{N} \sum_{i=1}^N (1 -\frac{m_i}{M_i})$$
    with $N$ the number of instances, $m_i$ the number of nodes in the important subgraph used for the explanation at instance $i$ and $M_i$ the number of nodes of the input graph at instance $i$.

In [14]:
import os
import numpy as np

EXPLAINED_DATA_DIR = os.path.join(BASE_DATA_DIR, 'explained')
os.makedirs(EXPLAINED_DATA_DIR, exist_ok=True)


In [16]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the training set...')
x_train_explained, y_train_explained, train_scores = get_all_explanations(
    x_train,
    y_train,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=REMOVE_VALUE,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train.npy'), x_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train.npy'), y_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'train_scores.npy'), train_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train_time.npy'), x_train_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train_time.npy'), y_train_time)


Computing the explanations for the training set...
[999/999] - 6378s - MAE: { severe_congestion 3.83 -congestion 1.92 -free_flow 0.804 - total: 2.18 } - RMSE: { severe_congestion 4.34 -congestion 2.14 -free_flow 0.93 - total: 2.46 } - MAPE: { severe_congestion 16.7% -congestion 3.88% -free_flow 1.22% - total: 7.24% } - Average time: 6.38s 


In [17]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the validation set...')
x_val_explained, y_val_explained, val_scores = get_all_explanations(
    x_val,
    y_val,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=REMOVE_VALUE,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val.npy'), x_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val.npy'), y_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'val_scores.npy'), val_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val_time.npy'), x_val_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val_time.npy'), y_val_time)

Computing the explanations for the validation set...
[198/198] - 1314s - MAE: { severe_congestion 2.95 -congestion 2.21 -free_flow 0.77 - total: 1.93 } - RMSE: { severe_congestion 3.32 -congestion 2.5 -free_flow 0.888 - total: 2.18 } - MAPE: { severe_congestion 12.4% -congestion 4.83% -free_flow 1.18% - total: 5.99% } - Average time: 6.64s 


In [18]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the test set...')
x_test_explained, y_test_explained, test_scores = get_all_explanations(
    x_test,
    y_test,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=REMOVE_VALUE,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test.npy'), x_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test.npy'), y_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'test_scores.npy'), test_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test_time.npy'), x_test_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test_time.npy'), y_test_time)

Computing the explanations for the test set...
[300/300] - 1920s - MAE: { severe_congestion 4.05 -congestion 2.22 -free_flow 0.725 - total: 2.32 } - RMSE: { severe_congestion 4.41 -congestion 2.45 -free_flow 0.837 - total: 2.55 } - MAPE: { severe_congestion 16.8% -congestion 4.64% -free_flow 1.1% - total: 7.47% } - Average time: 6.4s 


In [16]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_train.npy'))

x_val_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_val.npy'))

x_test_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test.npy'))

In [18]:
from src.explanation.monte_carlo.explanation import print_all_fidelity_plus


print('Computing the fidelity+ for the training set...')
print_all_fidelity_plus(x_train, y_train, x_train_explained, spatial_temporal_gnn, scaler, remove_value=REMOVE_VALUE)
print()

print('Computing the fidelity+ for the validation set...')
print_all_fidelity_plus(x_val, y_val, x_val_explained, spatial_temporal_gnn, scaler, remove_value=REMOVE_VALUE)
print()

print('Computing the fidelity+ for the test set...')
print_all_fidelity_plus(x_test, y_test, x_test_explained, spatial_temporal_gnn, scaler, remove_value=REMOVE_VALUE)
print()

Computing the fidelity+ for the training set...
MAE+: { severe_congestion 9.94 -congestion 28.2 -free_flow 26.3 - total: 21.5% } -RMSE+: { severe_congestion 11 -congestion 29.5 -free_flow 28.9 - total: 23.1% } -MAPE+: { severe_congestion 39.4% -congestion 53% -free_flow 40.3% - total: 44.2% }

Computing the fidelity+ for the validation set...
MAE+: { severe_congestion 9.65 -congestion 25.7 -free_flow 24.8 - total: 20% } -RMSE+: { severe_congestion 10.6 -congestion 26.9 -free_flow 28 - total: 21.8% } -MAPE+: { severe_congestion 37.8% -congestion 48.8% -free_flow 38% - total: 41.5% }

Computing the fidelity+ for the test set...
MAE+: { severe_congestion 10.7 -congestion 27.3 -free_flow 27.9 - total: 22% } -RMSE+: { severe_congestion 11.7 -congestion 28.5 -free_flow 30.8 - total: 23.7% } -MAPE+: { severe_congestion 41.2% -congestion 51.6% -free_flow 42.6% - total: 45.1% }



In [20]:
from src.explanation.monte_carlo.explanation import print_all_sparsity


print('Computing the sparsity for the training set...')
print_all_sparsity(x_train, x_train_explained)
print()

print('Computing the sparsity for the validation set...')
print_all_sparsity(x_val, x_val_explained)
print()

print('Computing the sparsity for the test set...')
print_all_sparsity(x_test, x_test_explained)
print()

Computing the sparsity for the training set...
Sparsity: { severe_congestion 0.989 -congestion 0.992 -free_flow 0.989 - total: 0.99 }

Computing the sparsity for the validation set...
Sparsity: { severe_congestion 0.99 -congestion 0.991 -free_flow 0.987 - total: 0.989 }

Computing the sparsity for the test set...
Sparsity: { severe_congestion 0.99 -congestion 0.992 -free_flow 0.989 - total: 0.99 }

